# AdaBoost and XGBoost models on the loan-level delinquency rates

Explanation
asdf
asdf
a
sdf

asdf

df

asdf

asdf

## Dataset cleaning

Create 4 datasets with 4 different MEI projected values

## Feature Engineering

Sort out categorical variables and other non-numerical

## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd
import numpy as np

# Assuming agg_loan is your dataframe and it's already loaded
# Let's say the 'D90' column is your binary target and the rest are features

# Splitting the data into features and target
X = agg_loan.drop('D90', axis=1)
y = agg_loan['D90']

# Splitting the dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# AdaBoost does not inherently have a feature selection mechanism like XGBoost,
# so to "force" a feature like DHRI, you might consider feature engineering techniques
# or include it in every set of hyperparameters if it was categorical. Since it's not,
# AdaBoost will naturally give it a weight in the model training process.

# Create the AdaBoost model
model = AdaBoostClassifier()

# Hyperparameter tuning with cross-validation
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1.0]
}

# GridSearchCV with the specified parameter grid
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='f1', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best hyperparameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the final model with the best parameters
final_model = AdaBoostClassifier(**best_params)
final_model.fit(X_train, y_train)

# Predictions
y_pred = final_model.predict(X_test)

# Performance metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'ROC AUC Score: {roc_auc}')


## XGBoost

Start with XGBoost model

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd
import numpy as np
import warnings
# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
# Assuming agg_loan_xgb is your dataframe and it's already loaded
# Let's say the 'D90' column is your binary target and the rest are features

# Splitting the data into features and target
X = agg_loan_xgb.drop('D90', axis=1)
y = agg_loan_xgb['D90']

# Splitting the dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Create the XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', enable_categorical=True)

# Hyperparameter tuning with cross-validation
param_grid = {
    'max_depth': [3, 4, 5, 10, 20],
}

# Including "DHRI" in the model; assume that "DHRI" is continuous.
X_train['DHRI'] = X_train['DHRI'].astype(float)


# GridSearchCV with the specified parameter grid
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='f1', verbose=2, n_jobs=-1)
grid_search.fit(
    X_train, 
    y_train  # Call the callback function every 50 rounds
)

# Best hyperparameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Train the final model with the best parameters
final_model = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
final_model.fit(X_train, y_train)

# Predictions
y_pred = final_model.predict(X_test)

# Performance metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'ROC AUC Score: {roc_auc}')


## Performance Metrics

### Confusion Matrix

### F1-Score